In support of the World Bank's ongoing support to the CoVID response in Africa, the INFRA-SAP team has partnered with the Chief Economist of HD to analyze the preparedness of the health system to respond to CoVID, focusing on ideas around infrastructure: access to facilities, demographics, electrification, and connectivity.

https://github.com/worldbank/INFRA_SAP/wiki/Kenya-CoVID-response

In [1]:
import os, sys, importlib
import rasterio, affine, gdal

import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import skimage.graph as graph

from shapely.geometry import Point, shape, box
from shapely.wkt import loads
from shapely.ops import cascaded_union
from rasterio import features

import GOSTnets as gn
import GOSTnets.load_osm as losm

sys.path.append("/home/wb411133/Code/GOST")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc
import GOSTRocks.Urban.UrbanRaster as urban

sys.path.append("../")
import infrasap.market_access as ma

In [2]:
country = "KEN"
iso2 = "KE"
out_folder = "/home/wb411133/data/Country/%s" % country
travel_folder = os.path.join(out_folder, 'TRAVEL_TIMES')

road_network = "/home/wb411133/data/Country/KEN/INFRA/KEN_OSM_OSMLR_1_3.osm.pbf"
network_map = os.path.join(travel_folder, "road_network.tif")

all_hospitals = "/home/public/Data/COUNTRY/KEN/HD_INF/merged_hospitals.shp"
critical_facilities = "/home/wb411133/data/Country/KEN/INFRA/HEALTH_INF/KEN_Critical_Care_Facilities.csv"

pop_layer = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/MOSAIC_ppp_prj_2020/ppp_prj_2020_%s.tif" % country
vul_map = "/home/public/Data/COUNTRY/KEN/HD_INF/KEN_vulnerability_map.tif"
urbanPop = os.path.join(out_folder, "wp_urban_pop.tif")

gsm_folder = "/home/public/Data/GLOBAL/INFRA/GSMA/2019/MCE/Data_MCE/Global"
gsm_2g = os.path.join(gsm_folder, "MCE_Global2G_2020.tif")
gsm_3g = os.path.join(gsm_folder, "MCE_Global3G_2020.tif")
gsm_4g = os.path.join(gsm_folder, "MCE_Global4G_2020.tif")

energy_folder = "/home/public/Data/COUNTRY/%s/GEP" % country
energy_settlements = os.path.join(energy_folder, "Kenya_final_clusters.shp")
energy_scenario = os.path.join(energy_folder, "ke-1-1_1_1_1_1_0.csv")

in_wards = "/home/public/Data/COUNTRY/KEN/ADMIN/KEN_adm4.shp"

inH = gpd.read_file(all_hospitals)
inW = gpd.read_file(in_wards)
pop_data = rasterio.open(pop_layer)

if not os.path.exists(critical_facilities.replace(".csv",".shp")):
    critical_h = pd.read_csv(critical_facilities)
    c_geom = [Point(x) for x in zip(critical_h['Long'], critical_h['Lat'])]
    in_c = gpd.GeoDataFrame(critical_h, geometry=c_geom, crs={'init':'epsg:4326'})
    in_c.to_file(critical_facilities.replace(".csv",".shp"))
else:
    in_c = gpd.read_file(critical_facilities.replace(".csv",".shp"))

# Clean Data

In [3]:
bad_facilities = ["VCT Centre","Blood Centre","Facility Type","Radiology Unit","Hospice"]
inH = inH.loc[~inH['type'].isin(bad_facilities)]
bedH = inH.loc[inH['Beds'] > 0]
onlyH = inH.loc[inH['type'] == "Hospital"]
inH['type'].value_counts()

Dispensary                4664
Clinic                    3181
Health Centre             1145
Hospital                   584
Maternity/Nursing Home     220
Laboratory                  57
Mobile Clinic               53
Health Programme            30
Pharmacy                    13
Name: type, dtype: int64

# Measure access to facilitites

In [4]:
# Create the traversal time map
if not os.path.exists(network_map):
    loadOSM = losm.OSM_to_network(road_network)
    loadOSM.generateRoadsGDF()
    roads = loadOSM.roadsGPD
    roads['speed'] = roads['infra_type'].map(ma.speed_dict)
    roads['geometry'] = roads['Wkt']
    traversal_time = ma.generate_network_raster(pop_data, roads)
    meta = pop_data.meta.copy()
    meta.update(dtype=traversal_time.dtype)

    with rasterio.open(network_map, 'w', **meta) as outR:
        outR.write_band(1, traversal_time)
else:
    network_r = rasterio.open(network_map)
    meta = network_r.meta.copy()
    traversal_time = network_r.read()[0,:,:]

In [5]:
mcp = graph.MCP_Geometric(traversal_time)    

In [6]:
# calculate minimum travel time to nearest facility with a bed
travel_time = os.path.join(travel_folder, "bed_tt.tif")
if not os.path.exists(travel_time):
    dests = list(set([pop_data.index(x.x, x.y) for x in bedH['geometry']]))
    dests = [(d) for d in dests if (d[0] > 0 and d[1] > 0)]
    costs, traceback = mcp.find_costs(dests)    
    costs = costs.astype(pop_data.meta['dtype'])
    with rasterio.open(travel_time, 'w', **pop_data.meta) as out_f:
        out_f.write_band(1, costs)

In [7]:
# calculate minimum travel time to nearest facility
travel_time = os.path.join(travel_folder, "all_facilities_tt.tif")
if not os.path.exists(travel_time):
    dests = list(set([pop_data.index(x.x, x.y) for x in good_f['geometry']]))
    dests = [(d) for d in dests if (d[0] > 0 and d[1] > 0)]
    dests = [(d) for d in dests if (d[0] <= traversal_time.shape[0] and d[1] <= traversal_time.shape[1])]
    costs, traceback = mcp.find_costs(dests)    
    costs = costs.astype(pop_data.meta['dtype'])
    with rasterio.open(travel_time, 'w', **pop_data.meta) as out_f:
        out_f.write_band(1, costs)

In [8]:
# calculate minimum travel time to critical care facility
travel_time = os.path.join(travel_folder, "cc_facilities_tt.tif")
if not os.path.exists(travel_time):
    dests = list(set([pop_data.index(x.x, x.y) for x in in_c['geometry']]))
    dests = [(d) for d in dests if (d[0] > 0 and d[1] > 0)]
    dests = [(d) for d in dests if (d[0] <= traversal_time.shape[0] and d[1] <= traversal_time.shape[1])]
    costs, traceback = mcp.find_costs(dests)    
    costs = costs.astype(pop_data.meta['dtype'])
    with rasterio.open(travel_time, 'w', **pop_data.meta) as out_f:
        out_f.write_band(1, costs)

In [11]:
# calculate minimum travel time to nearest hospital
travel_time = os.path.join(travel_folder, "hospital_tt.tif")
if not os.path.exists(travel_time):
    dests = list(set([pop_data.index(x.x, x.y) for x in inH['geometry']]))
    dests = [(d) for d in dests if (d[0] > 0 and d[1] > 0)]
    costs, traceback = mcp.find_costs(dests)    
    costs = costs.astype(pop_data.meta['dtype'])
    with rasterio.open(travel_time, 'w', **pop_data.meta) as out_f:
        out_f.write_band(1, costs)

# Summarize Wards

In [12]:
# Population zonal stats
res = rMisc.zonalStats(inW, pop_layer, minVal=0, allTouched = True)
res = pd.DataFrame(res, columns=["SUM","MIN",'MAX',"SUM"])

inW['Pop'] = 0
inW['Pop'] = res['SUM']

In [13]:
# Calculate urban population
urb_calculator = urban.urbanGriddedPop(pop_layer)
if not os.path.exists(urbanPop):
    urban_res = urb_calculator.calculateUrban(densVal = 3, totalPopThresh=5000,
                              smooth=False, raster_pop = urbanPop)
res = rMisc.zonalStats(inW, urbanPop, minVal=0, allTouched = True)
res = pd.DataFrame(res, columns=["SUM","MIN",'MAX',"SUM"])
inW['URB_POP'] = 0
inW['URB_POP'] = res['SUM']

In [14]:
#Summarize vulnerable population
res = rMisc.zonalStats(inW, vul_map, minVal=0)
res = pd.DataFrame(res, columns=['SUM','MIN','MAX','MEAN'])
inW["VUL_POP"] = res['SUM']

In [15]:
# Summarize Population within driving times
in_files = ['bed_tt.tif','hospital_tt.tif','cc_facilities_tt.tif','all_facilities_tt.tif']
pop_data = rasterio.open(pop_layer).read()[0,:,:]

for tt_file in in_files:
    tt = rasterio.open(os.path.join(travel_folder, tt_file))
    tt_data = tt.read()[0,:,:]
    for min_thresh in [1800, 3600, 7200, 14400]:
        out_file = os.path.join(travel_folder, tt_file.replace(".tif", "_%s_pop.tif" % (min_thresh)))
        if not os.path.exists(out_file):
            tt_thresh = (tt_data < (min_thresh)).astype(int)
            thresh_pop = tt_thresh * pop_data
            thresh_pop = thresh_pop.astype(tt.meta['dtype'])
            with rasterio.open(out_file, 'w', **tt.meta) as outR:
                outR.write_band(1, thresh_pop)
        res = rMisc.zonalStats(inW, out_file, minVal=0, allTouched = True)
        res = pd.DataFrame(res, columns=["SUM","MIN",'MAX',"SUM"])
        column_name = "%s_%s" %  (tt_file.split("_")[0], min_thresh)
        inW[column_name] = 0
        inW[column_name] = res['SUM']
        misc.tPrint("%s : %s" % (column_name, min_thresh))
        

09:34:23	bed_1800 : 1800
09:34:29	bed_3600 : 3600
09:34:35	bed_7200 : 7200
09:34:41	bed_14400 : 14400
09:34:48	hospital_1800 : 1800
09:34:54	hospital_3600 : 3600
09:35:00	hospital_7200 : 7200
09:35:06	hospital_14400 : 14400
09:35:12	cc_1800 : 1800
09:35:18	cc_3600 : 3600
09:35:24	cc_7200 : 7200
09:35:30	cc_14400 : 14400
09:35:36	all_1800 : 1800
09:35:42	all_3600 : 3600
09:35:48	all_7200 : 7200
09:35:54	all_14400 : 14400


In [16]:
inW.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,ID_3,NAME_3,ID_4,...,hospital_7200,hospital_14400,cc_1800,cc_3600,cc_7200,cc_14400,all_1800,all_3600,all_7200,all_14400
0,119,KEN,Kenya,1523,Nyanza,18968,Homa Bay,36311,Kendul Bay,18498,...,31739.177734,31739.177734,0.000000,0.000000,9512.117188,31739.177734,31409.388672,31739.177734,31739.177734,31739.177734
1,119,KEN,Kenya,1523,Nyanza,18968,Homa Bay,36311,Kendul Bay,18499,...,41396.855469,41396.855469,0.000000,0.000000,40910.492188,41396.855469,41349.046875,41396.855469,41396.855469,41396.855469
2,119,KEN,Kenya,1523,Nyanza,18968,Homa Bay,36311,Kendul Bay,18500,...,30115.062500,30115.062500,0.000000,0.000000,175.949783,30115.062500,29261.164062,30115.062500,30115.062500,30115.062500
3,119,KEN,Kenya,1523,Nyanza,18968,Homa Bay,36312,Lake Victoria,18501,...,55619.367188,60611.066406,3190.292969,4819.221191,17660.230469,49136.664062,57835.078125,60569.410156,60611.066406,60611.066406
4,119,KEN,Kenya,1523,Nyanza,18968,Homa Bay,36313,Maseno,18502,...,-1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000


In [17]:
# create risk index based on hospital access, urbanization, and vulnerability
inW['VUL_IDX'] = (inW['VUL_POP'].rank() > (inW.shape[0] * 0.8)).astype(int)
inW['URB_IDX'] = (inW['URB_POP'] / inW['Pop']) > 0.7
inW['ACC_IDX'] = (inW['all_3600'] / inW['Pop']) > 0.5

In [18]:
in_settlements = gpd.read_file(energy_settlements)
in_scenario = pd.read_csv(energy_scenario)
in_settlements = pd.merge(in_settlements, in_scenario, on="id")
settlement_index = in_settlements.sindex

In [19]:
inW['sPop'] = 0
inW['ePop'] = 0
inW['ePop2'] = 0
for idx, row in inW.iterrows():
    # find possible settlements
    possible_matches_index = list(settlement_index.intersection(row['geometry'].bounds))
    possible_matches = in_settlements.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(row['geometry'])]
    inW.loc[idx, 'sPop'] = precise_matches['Population'].sum()
    inW.loc[idx, 'ePop'] = precise_matches['ElecPop_x'].sum()
    inW.loc[idx, 'ePop2'] = precise_matches.loc[precise_matches['ElecStart'] == 1, 'Population'].sum()
    

In [20]:
inW.to_file(os.path.join(out_folder, "Wards.shp"))

# Summarizing facilities

In [21]:
sindex = inW.sindex
#attach WARD ID to the facilities
inH['MYWARD'] = 0
allVals = []
for idx, row in inH.iterrows():
    selW = inW.loc[list(sindex.nearest((row['geometry'].x, row['geometry'].y)))[0]]
    try:
        inH.loc[idx, "MYWARD"] = selW['ID_4']
    except:
        break

In [24]:
# summarize facility access to other facilities (CC, hospitals, and all facilities)
for raster_defs in [
                    ['BED_ACC', 'bed_tt.tif'],
                    ['CCF_ACC', "cc_facilities_tt.tif"],
                    ['HOS_ACC', "hospital_tt.tif"],                    
                   ]:
    cc_tt = rasterio.open(os.path.join(travel_folder, raster_defs[1]))
    inH[raster_defs[0]] = 0
    for idx, row in inH.iterrows():
        try:
            val = list(cc_tt.sample([(row['geometry'].x, row['geometry'].y)]))[0][0]
        except:
            val = 0
        inH.loc[idx, raster_defs[0]] = val

In [25]:
# Attribute facilities with GSM coverage
inR_2g = rasterio.open(gsm_2g)
inR_3g = rasterio.open(gsm_3g)
inR_4g = rasterio.open(gsm_4g)
inH = inH.to_crs(inR_2g.crs)

inH_pts = [[x.x, x.y] for x in inH['geometry']]
vals = [x[0] for x in list(inR_2g.sample(inH_pts))]
print(sum(vals))
inH.loc[:,'gsm2g'] = 0
inH.loc[:,'gsm2g'] = vals

vals = [x[0] for x in list(inR_3g.sample(inH_pts))]
print(sum(vals))
inH.loc[:,'gsm3g'] = 0
inH.loc[:,'gsm3g'] = vals

vals = [x[0] for x in list(inR_4g.sample(inH_pts))]
print(sum(vals))
inH.loc[:,'gsm4g'] = 0
inH.loc[:,'gsm4g'] = vals

inH = inH.to_crs({'init':'epsg:4326'})

11714
24895
29841


In [29]:
# Attribute facilities with electrification
inH['sPop'] = 0
inH['sEPop'] = 0
inH['sElec'] = 0

inH = inH.to_crs(in_settlements.crs)
for idx, row in inH.iterrows():
    # identify settlement and store electrification information
    select_settlement = in_settlements.loc[list(settlement_index.nearest([row['geometry'].x, row['geometry'].y]))]        
    pop = select_settlement['Population'].values[0]
    elecPop = select_settlement['ElecPop_y'].values[0]
    startElec = select_settlement['ElecStart'].values[0]
    inH.loc[idx, 'sPop'] = pop
    inH.loc[idx, 'sEPop'] = elecPop
    inH.loc[idx, 'sElec'] = startElec
inH = inH.to_crs({'init':'epsg:4326'})

In [31]:
inH.to_file(os.path.join(out_folder, "attributed_hospitals.shp"))